# Spark ML

### Code to be executed before lecture

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

In [ ]:
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

### WARNING: do not keep multiple copies of `sf.csv` on your VM as it will eat up disk space

Please make sure to move sf.csv into `25-spark-ml/nb` directory.

In [ ]:
!hdfs dfs -cp sf.csv hdfs://nn:9000/sf.csv

In [ ]:
df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .load("hdfs://nn:9000/sf.csv"))

In [ ]:
cols = [col(c).alias(c.replace(" ", "_")) for c in df.columns]
df.select(cols).write.mode("ignore").format("parquet").save("hdfs://nn:9000/sf.parquet")

In [ ]:
!hdfs dfs -rm hdfs://nn:9000/sf.csv

In [ ]:
(spark.read
 .format("parquet")
 .load("hdfs://nn:9000/sf.parquet")
 .createOrReplaceTempView("calls")
)

### Lecture starts here

### Spark execution explanation
`.explain()` or `.explain("formatted")`

In [ ]:
spark.sql("""
SELECT Call_Type, COUNT(*)
FROM calls
GROUP BY Call_Type
""").explain("formatted")

### Bucketed data

`bucketBy(<numBuckets>, <col>)`

In [ ]:
# would work without sampling, just using it to make it faster
(spark.table("calls")
 .sample(True, 0.01)
 .write
 .mode("overwrite")
 .bucketBy(10, "Call_Type")
 .saveAsTable("call_by_type"))

Let's repeat the same SQL query now.

In [ ]:
spark.sql("""
SELECT Call_Type, COUNT(*)
FROM call_by_type
GROUP BY Call_Type
""").explain("formatted")

### JOIN Algorithms (for a single machine)

In [ ]:
# kind_id, color
fruits = [
    ("B", "Yellow"),
    ("A", "Green"),
    ("C", "Orange"),
    ("A", "Red"),
    ("C", "Purple"),
    ("B", "Green")
]

# kind_id, name (assume no duplicate kind_id's)
kinds = [
    ("A", "Apple"),
    ("B", "Banana"),
    ("C", "Carrot")
]

#### GOAL: print Yellow Banana, Green Apple, etc (any order)

### Option 1: Hash join
- Move smaller table to in-memory Python `dict`
- Iterate over larger table and find matches using `dict` lookup

In [ ]:
kind_lookup = dict(kinds)
kind_lookup

In [ ]:
for kind_id, color in fruits:
    print(color, kind_lookup[kind_id])

### Option 2: sort merge join

- Sort both tables (can be done using disk too)
- Iterate over smaller table
  - Conditionally iterate over bigger table to find matches

In [ ]:
fruits.sort()
kinds.sort()

In [ ]:
fruits

In [ ]:
kinds

In [ ]:
fruit_idx = 0
for kind_id, food_name in kinds:
    while fruit_idx < len(fruits):
        if fruits[fruit_idx][0] > kind_id:
            break
        elif fruits[fruit_idx][0] == kind_id:
            print(fruits[fruit_idx][1], food_name)
        fruit_idx += 1

### Spark ML

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame({"x1": np.random.randint(0, 10, 100).astype(float), 
                   "x2": np.random.randint(0, 3, 100).astype(float)})
df["y"] = df["x1"] + df["x2"] + np.random.rand(len(df))

Let's convert pandas dataframe to Spark dataframe.

In [ ]:
df = spark.createDataFrame(df)
df

Recall that seed in Spark is not truly deterministic overall (because everytime we might have new partitions), just deterministic at the partition level.

In [ ]:
train, test = df.randomSplit([0.75, 0.25], seed=42)
test.show()

Let's write data to Parquet format and read the data from the Parquet file.
We need to now use `mode("ignore")` to make sure that we work with the deterministic sample.

In [ ]:
train.write.format("parquet").mode("ignore").save("hdfs://nn:9000/train.parquet")
test.write.format("parquet").mode("ignore").save("hdfs://nn:9000/test.parquet")

In [ ]:
train = spark.read.format("parquet").load("hdfs://nn:9000/train.parquet")
test = spark.read.format("parquet").load("hdfs://nn:9000/test.parquet")

In [ ]:
train.count(), test.count()

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor, DecisionTreeRegressionModel

- `DecisionTreeRegressor`: unfit model
- `DecisionTreeRegressionModel`: fitted model
    - In Spark, names ending in "Model" are the fitted ones

In [ ]:
# ALWAYS needs a vector column - even for a single feature!
# dt = DecisionTreeRegressor(featuresCol="x1", labelCol="y")
# dt.fit(train)

### VectorAssembler

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
va = VectorAssembler(inputCols=["x1", "x2"], outputCol="features")
va.transform(train).show()

In [ ]:
va = VectorAssembler(inputCols=["x1", "x2"], outputCol="features")
dt = DecisionTreeRegressor(featuresCol="features", labelCol="y")

model = dt.fit(va.transform(train))

In [ ]:
type(dt), type(model)